In [1]:
import spacy
import os 
from pprint import pprint
import json 
import pandas as pd 
from tqdm import tqdm 

2023-09-21 02:06:48.707045: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-21 02:06:48.753722: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-21 02:06:49.565520: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-09-21 02:06:50.583372: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at htt

In [2]:
def mkdir(idir):
    if not os.path.isdir(idir):
        os.makedirs(idir)

def get_noun_phrases(doc, output=None, keep=None):
    if keep is None:
        return list([np.text.lower() for np in doc.noun_chunks])
    if output is None:
        output = {}
    kws = []
    for nc in doc.noun_chunks:
        ws = []
        for word in nc:
            if word.pos_ in keep:
                ws.append(word.text.lower())
        if len(ws) > 0:
            n = ' '.join(ws)
            output[n] = output.get(n, 0) + 1
            kws.append(n)
    return output, kws 

def increase_count(idict, key, freq):
    if key not in idict:
        idict[key] = 0
    idict[key] += freq


def add_to_dict(idict, key, value, freq=1):
    if key not in idict:
        idict[key] = {}
    if value not in idict[key]:
        idict[key][value] = 0
    idict[key][value] += freq


def get_unique_values(idict, count_only=False):
    if count_only:
        return {k: len(set(v)) for k, v in idict.items()}
    else:
        return {k: list(set(v)) for k, v in idict.items()}


def save_np_info(np2count, np2reviews, np2rest, np2users, ofile, count_only=False):
    # output = {"np2count": np2count, "np2review_count": count_unique_values(np2reviews), 
    #         'np2res_count': count_unique_values(np2rest), 'np2user_count': count_unique_values(np2users)}
    output = {"np2count": np2count, "np2reviews": np2reviews, 
            'np2rests': np2rest, 'np2users': np2users}
    json.dump(output, open(ofile, 'w'))
    print("Saved to", ofile)

def extract_raw_keywords_for_reviews(data, ofile, 
                                     keep=['ADJ', 'NOUN', 'PROPN', 'VERB'], overwrite=False, 
                                     review2keyword_ofile=None):
    if os.path.isfile(ofile) and not overwrite:
        print("Existing output file. Stop! (set overwrite=True to overwrite)")
        return 
    np2count = {}   # frequency 
    np2review2count = {}  # reviews 
    np2rest2count = {}  # 
    np2user2count = {} 
    counter = 0
    review2keywords = {}
    for rid, uid, restid, text in tqdm(zip(data['review_id'], data['user_id'], data['rest_id'], data['text']), total=len(data)):
        doc = nlp(text)
        tmp, keywords = get_noun_phrases(doc, keep=keep)  # np for this review 
        for np, freq in tmp.items():
            increase_count(np2count, np, freq)
            add_to_dict(np2review2count, np, rid, freq)
            add_to_dict(np2rest2count, np, restid, freq)
            add_to_dict(np2user2count, np, uid, freq)
        review2keywords[rid] = keywords
        # counter += 1
        # if counter % 2 == 0:
            # save_np_info(np2count, np2review2count, np2rest2count, np2user2count, ofile)
    save_np_info(np2count, np2review2count, np2rest2count, np2user2count, ofile)
    if review2keyword_ofile is not None: 
        df = pd.DataFrame({"Review_ID": list(review2keywords.keys()), "Keywords": list(review2keywords.values())})
        df.to_csv(review2keyword_ofile)


def load_split(sfile='/home/ubuntu/duc.nm195858/keyext_LLM/splits.json', city='singapore', setname='train'):
    return json.load(open(sfile))[city][setname]

### Extract keywords for train set
- Extract keywords from train set 
- Then use postprocessing for choosing a subset of keywords 
- Use these keywords for test, dev? (maybe don't need, but can do)

In [3]:
nlp = spacy.load("en_core_web_sm")

In [4]:
# setname = 'test'
CITIES = ['charlotte', 'edinburgh', 'lasvegas', 'london', 'phoenix', 'pittsburgh', 'singapore']
sets = ['train']#, 'test', 'dev']
for city in CITIES: 
    dt = pd.read_csv('/home/ubuntu/duc.nm195858/keyext_LLM/reviews/{}.csv'.format(city))
    for setname in sets: 
        print("Processing for", city, setname)
        uids = load_split(city=city, setname=setname)
        dt_set = dt[dt['user_id'].isin(uids)]
        odir = '/home/ubuntu/duc.nm195858/keyext_LLM/preprocessed/by_city-users_min_3_reviews/keywords_spacy/' + setname
        mkdir(odir)
        extract_raw_keywords_for_reviews(dt_set, ofile=os.path.join(odir, city + '-keywords.json'), keep=['ADJ', 'NOUN', 'PROPN', 'VERB'], 
                                        overwrite=True, review2keyword_ofile=os.path.join(odir,city+"-review2keywords.csv"))

Processing for charlotte train


100%|██████████| 90426/90426 [27:50<00:00, 54.14it/s]


Saved to /home/ubuntu/duc.nm195858/keyext_LLM/preprocessed/by_city-users_min_3_reviews/keywords_spacy/train/charlotte-keywords.json
Processing for edinburgh train


100%|██████████| 10342/10342 [03:47<00:00, 45.40it/s]


Saved to /home/ubuntu/duc.nm195858/keyext_LLM/preprocessed/by_city-users_min_3_reviews/keywords_spacy/train/edinburgh-keywords.json
Processing for lasvegas train


...

Saved to /home/ubuntu/duc.nm195858/keyext_LLM/preprocessed/by_city-users_min_3_reviews/keywords_spacy/train/phoenix-keywords.json
Processing for pittsburgh train


100%|██████████| 73558/73558 [22:25<00:00, 54.65it/s]


Saved to /home/ubuntu/duc.nm195858/keyext_LLM/preprocessed/by_city-users_min_3_reviews/keywords_spacy/train/pittsburgh-keywords.json
Processing for singapore train


100%|██████████| 10615/10615 [03:53<00:00, 45.51it/s]


Saved to /home/ubuntu/duc.nm195858/keyext_LLM/preprocessed/by_city-users_min_3_reviews/keywords_spacy/train/singapore-keywords.json
